# Detecting straight from dataset

## Libraries

In [1]:
import os
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import Augmentor

%matplotlib inline

# Libraries for TensorFlow
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers
from tensorflow.keras.models import clone_model
from tensorflow.keras.applications import resnet50
from tensorflow.keras import backend as K

from tensorflow.keras.applications.resnet50 import preprocess_input

import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from keras import backend as K

In [2]:
print("Num GPUs Available: ", tf.config.list_physical_devices())
K.image_data_format()

Num GPUs Available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2022-03-31 12:58:16.864592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/psolsona/anaconda3/envs/TFG_Project/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda-8.0/lib64:/usr/local/cuda-8.0/lib64::/usr/local/cuda-11.5/lib64
2022-03-31 12:58:16.864611: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


'channels_last'

In [3]:
Labels = ['File','Front', 'Reversed']
ReverseDownPath = r'Data\OriginalPantsDataset\ReverseDown'
ReverseUpPath = r'Data\OriginalPantsDataset\ReverseUp'
RightDownPath = r'Data\OriginalPantsDataset\RightDown'
RightUpPath = r'Data\OriginalPantsDataset\RightUp'
AllPaths = [ReverseDownPath,ReverseUpPath,RightDownPath,RightUpPath]
originalFilename = r'Data\OriginalPantsDataset\pants_dataset.csv'

## Creating CSV Images classification

In [4]:
allFiles = []
for i, path in enumerate(AllPaths):
    if(i == 0):
        Reverse = True
        Front = False
    elif i == 1:
        Reverse = True
        Front = True
    elif i == 2:
        Reverse = False
        Front = False
    elif i == 3:
        Reverse = False
        Front = True
    
    for files in os.listdir(path):
        allFiles.append([files,Front, Reverse])

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\OriginalPantsDataset\\ReverseDown'

In [ ]:
with open(originalFilename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(Labels) 
    csvwriter.writerows(allFiles)

## Data augmentation

Let's create more data from the existing one. We want to load images, compress them, make 100 variations of each, and save them

In [3]:
augmentedDataSetPath = r'PantsDataset'

In [ ]:
def saveImage(image, folderName, iPath, iRange, iFile):
    name = 'P_'+str(iPath)+str(iFile)+str(iRange)+'.jpg'
    if not os.path.exists(augmentedDataSetPath):
        os.mkdir(augmentedDataSetPath)
    finalDest = os.path.join(augmentedDataSetPath,folderName)
    if not os.path.exists(finalDest):
        os.mkdir(finalDest)
    finalDest = os.path.join(finalDest,name)

    image.save(finalDest, "JPEG", optimize = True, quality = 3)

In [9]:
allNewFiles = []
for iPath, path in enumerate(AllPaths):
    if(iPath == 0):
        pathName = "Reverse_Back"
    elif(iPath == 1):
        pathName = "Reverse_Front"
    elif(iPath == 2):
        pathName = "Alright_Back"
    elif(iPath == 3):
        pathName = "Alright_Front"
        
    for iFile, file in enumerate(os.listdir(path)):
        fullPath = os.path.join(path, file)
        image = Image.open(fullPath)
        saveImage(image, pathName,iPath, 0, iFile)
        print('Compressed ' + file)

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\OriginalPantsDataset\\ReverseDown'

# Making the generators

In [38]:
BATCH_SIZE = 128
EPOCHS = 20
WIDTH = 224
HEIGHT = 224
STEPS_EPOCH = 20

In [39]:
p = Augmentor.Pipeline(augmentedDataSetPath)
p.resize(1, WIDTH, HEIGHT)
p.flip_left_right(0.5)
p.rotate(probability=0.3, max_left_rotation=5, max_right_rotation=5)
p.skew(0.4, 0.5)#p.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5)

Initialised with 80 image(s) found.
Output directory set to PantsDataset/output.

In [40]:
p.status()

Operations: 4
	0: Resize (probability=1 width=224 height=224 resample_filter=BICUBIC )
	1: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	2: RotateRange (probability=0.3 max_left_rotation=-5 max_right_rotation=5 )
	3: Skew (probability=0.4 skew_type=RANDOM magnitude=0.5 )
Images: 80
Classes: 4
	Class index: 0 Class label: 0 
	Class index: 1 Class label: 1 
	Class index: 2 Class label: 2 
	Class index: 3 Class label: 3 
Dimensions: 1
	Width: 4032 Height: 2268
Formats: 1
	 JPEG

You can remove operations using the appropriate index and the remove_operation(index) function.


In [41]:
train_set = p.keras_generator(batch_size=BATCH_SIZE)
val_set = p.keras_generator(batch_size=BATCH_SIZE)

In [42]:
def checkPointCreation(checkpoint_path):
    print("saving checlpoint")
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    return cp_callback

In [43]:
def generateWeights(labels):
    class_weights = class_weight.compute_class_weight('balanced',classes = np.unique(labels),y=labels)
    leng = max(labels)
    class_weight_dict = {i : class_weights[i] for i in range(leng+1)}
    print(class_weight_dict)
    return class_weight_dict

In [44]:
def generateModel(classesAmount):
    input_layer = layers.Input(shape=(WIDTH,HEIGHT,3)) #We set the input layer to the data format we desire

    resNet=resnet50.ResNet50(weights='imagenet', input_tensor=input_layer,include_top=False) #We get the resnet model
    last_layer=resNet.output #We take output layers of resnet
    flatten=layers.Flatten()(last_layer) # Add flatten layer: we are extending Neural Network by adding flattn layer
    
    # Add dense layer to the final output layer
    output_layer = layers.Dense(classesAmount,activation='softmax')(flatten)

    # Creating model with input and output layer
    model=models.Model(inputs=input_layer,outputs=output_layer)
    model.summary()

    #for layer in model.layers[:-1]: #Freezing lower layers (in this case only lower layer)
    #    layer.trainable=False
    model.summary()
    return clone_model(model)

In [45]:
def load_weights(cp_path):
    print("Getting weights")
    current_Epoch = 0
    filepath = ''
    if os.path.exists(cp_path):
        for filename in os.listdir(cp_path):
            f = os.path.join(cp_path, filename)
            # checking if it is a file
            if os.path.isfile(f):
                try:
                    lsp = filename.split('-')[1]
                    num = int(lsp.split('.')[0])
                    if(current_Epoch <= num):
                        current_Epoch = num
                        file = filename.split('.index')[0]
                        filepath = os.path.join(cp_path, file)
                        print(filepath)
                except:
                    print("not this")
        print("Starting from epoch " + str(current_Epoch))
    else:
        os.mkdir(cp_path)
    return current_Epoch, filepath

In [46]:
"""
Function extracted from https://deeplizard.com/learn/video/km7pxKy4UHU to create a confusion matrix
"""
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# First Test

In [47]:
cp_path = "checkPoints/model/cp-{epoch:04d}.ckpt"
cp_callback = checkPointCreation(cp_path)

saving checlpoint


In [48]:
model = generateModel(len(p.class_labels))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [49]:
currentEpoch, file = load_weights("checkPoints/model")
if(currentEpoch != 0):
    model.load_weights(file)

Getting weights
Starting from epoch 0


In [51]:
model.fit(train_set,steps_per_epoch=STEPS_EPOCH,epochs=EPOCHS,initial_epoch = currentEpoch,verbose=True,callbacks=[cp_callback])

Epoch 1/20
20/20 [==============================] - 377s 19s/step - loss: 21.7346 - accuracy: 0.2660

Epoch 00001: saving model to checkPoints/model/cp-0001.ckpt
Epoch 2/20
20/20 [==============================] - 379s 19s/step - loss: 3.4330 - accuracy: 0.2730

Epoch 00002: saving model to checkPoints/model/cp-0002.ckpt
Epoch 3/20
20/20 [==============================] - 379s 19s/step - loss: 3.4851 - accuracy: 0.3078

Epoch 00003: saving model to checkPoints/model/cp-0003.ckpt
Epoch 4/20
20/20 [==============================] - 379s 19s/step - loss: 2.7559 - accuracy: 0.3668

Epoch 00004: saving model to checkPoints/model/cp-0004.ckpt
Epoch 5/20
20/20 [==============================] - 379s 19s/step - loss: 1.6390 - accuracy: 0.5305

Epoch 00005: saving model to checkPoints/model/cp-0005.ckpt
Epoch 6/20
20/20 [==============================] - 379s 19s/step - loss: 1.1240 - accuracy: 0.6711

Epoch 00006: saving model to checkPoints/model/cp-0006.ckpt
Epoch 7/20
20/20 [===============

KeyboardInterrupt: 

In [ ]:
if(currentEpoch < EPOCHS):
    model.save('saved_model/model')